In [32]:
import numpy as np
from os.path import exists
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import codecs


In [30]:
#file parser
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def stem_text(text):

   
    def filter_words(word):
        return not (word in stop_words)
    
    def is_correct(word):
        return len(word)>2 and word.isalpha()

        
    tokens = word_tokenize(text)
    word_limit_per_article = int(1e4) 
    words_list = []
    
    for i, word in enumerate(tokens):

        if filter_words(word) and is_correct(word):
            words_list.append(word)

        if i > word_limit_per_article:
            break



    for i, w in enumerate(words_list):
        words_list[i] = stemmer.stem(w)
        
    res = " ".join(words_list)
    
    return res



def parse_file(file_name, new_name):
    res = ""
    with codecs.open(file_name,'r', errors="ignore", encoding="utf-8") as f:
        for line in f.readlines():
            if not line.isspace():
                res += line

    res1 = stem_text(res)
    
    with codecs.open(new_name, 'w+', errors="ignore", encoding="utf-8") as f:
        f.write(res1)
    
    

def parse_files():
    directory = 'DIR' 
    parsed_files = 'parsed_files/'
    suffix = '.txt'

    for i, filename in enumerate(os.listdir(directory)):
        file_name = os.path.join(directory, filename)
        new_name = parsed_files + str(i)+"@"+filename

        if exists(file_name):
            parse_file(file_name, new_name)




In [31]:
parse_files()


Od teraz pliki nie posiadją pustych linii, a słowa są 'zestemowane' oraz odfiltrowane, tak aby nie były one 'stop wordami'.

In [36]:
import os
import re


def calulate_union():

    directory = 'parsed_files'
    res = dict()
    pattern = re.compile("^[a-zA-Z]+$")
    
    for i, filename in enumerate(os.listdir(directory)):
        f = os.path.join(directory, filename)

        if os.path.isfile(f):
            with open(f, 'r', errors="ignore") as file:
                for line in file.readlines():
                    for word in line.split():
                        if pattern.match(word):
                            res[word] = 1

    return set(res.keys())
set_of_words = calulate_union()
union_of_words = " ".join(set_of_words)
with open('union.txt', 'w+', errors="ignore") as f:
    f.write(union_of_words)

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
# class Search_Engine: 
#     def __init__(self, ):

dict_of_words = {word: i for i, word in enumerate(set_of_words)}
print(len(dict_of_words))

no_files = 45362

matrix = scipy.sparse.lil_matrix((len(set_of_words),no_files))
directory = 'parsed_files'

def extract_col(file_name):
    for i in range(len(file_name)):
        if file_name[i]=="@":
            rem = i
            break
    
    return int(file_name[:rem])

pattern = re.compile("^[a-zA-Z]+$")

for i, filename in enumerate(os.listdir(directory)):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        with open(f, "r",errors="ignore") as file:
            col = extract_col(filename)
            for line in file.readlines():
                for word in line.split():
                    if pattern.match(word):
                        matrix[dict_of_words[word],col] += 1

    if i >int(4e4):
        break     
    if i % 500 == 0:
        print((i//500), "% done")


985484
0 % done
1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done


  (253, 0)	1.0
  (4518, 0)	1.0
  (5101, 0)	1.0
  (5351, 0)	3.0
  (6036, 0)	1.0
  (7382, 0)	1.0
  (8458, 0)	1.0
  (9099, 0)	1.0
  (9387, 0)	1.0
  (11814, 0)	2.0
  (12191, 0)	1.0
  (12407, 0)	1.0
  (13553, 0)	2.0
  (15228, 0)	1.0
  (18627, 0)	3.0
  (18776, 0)	1.0
  (20766, 0)	1.0
  (20796, 0)	1.0
  (21451, 0)	19.0
  (21534, 0)	1.0
  (21672, 0)	2.0
  (24145, 0)	2.0
  (27117, 0)	1.0
  (27308, 0)	1.0
  (27710, 0)	3.0
  :	:
  (963222, 0)	1.0
  (964257, 0)	1.0
  (964668, 0)	2.0
  (964948, 0)	1.0
  (966289, 0)	2.0
  (967846, 0)	1.0
  (968019, 0)	1.0
  (969558, 0)	2.0
  (971269, 0)	1.0
  (971820, 0)	1.0
  (972033, 0)	1.0
  (972415, 0)	1.0
  (973023, 0)	1.0
  (973127, 0)	3.0
  (973600, 0)	1.0
  (973803, 0)	3.0
  (975087, 0)	1.0
  (976447, 0)	1.0
  (978938, 0)	11.0
  (979659, 0)	1.0
  (980982, 0)	1.0
  (983377, 0)	1.0
  (984341, 0)	1.0
  (985271, 0)	2.0
  (985272, 0)	3.0
august


In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
#"file_name": "countr vect"

main_dict = {}

directory = 'parsed_files'

for i, filename in enumerate(os.listdir(directory)):
    f = os.path.join(directory, filename)

    if os.path.isfile(f):
        with open(f, "r", errors="ignore") as file:
            main_dict[filename] = count_vect.fit_transform(file)



In [73]:
some_file_name = "14@Ambo,_Ethiopia.txt"



  (0, 288)	1
  (0, 725)	2
  (0, 629)	3
  (0, 38)	34
  (0, 229)	1
  (0, 708)	29
  (0, 554)	9
  (0, 264)	1
  (0, 486)	1
  (0, 39)	1
  (0, 756)	1
  (0, 265)	1
  (0, 267)	1
  (0, 555)	1
  (0, 652)	6
  (0, 579)	6
  (0, 244)	1
  (0, 52)	2
  (0, 163)	1
  (0, 753)	3
  (0, 175)	3
  (0, 451)	3
  (0, 556)	7
  (0, 40)	1
  (0, 263)	12
  :	:
  (0, 374)	1
  (0, 436)	1
  (0, 430)	1
  (0, 420)	1
  (0, 59)	1
  (0, 528)	1
  (0, 532)	2
  (0, 417)	1
  (0, 771)	1
  (0, 671)	1
  (0, 773)	1
  (0, 284)	1
  (0, 29)	1
  (0, 92)	1
  (0, 126)	1
  (0, 373)	1
  (0, 440)	1
  (0, 276)	1
  (0, 516)	1
  (0, 634)	1
  (0, 680)	1
  (0, 736)	1
  (0, 67)	1
  (0, 172)	1
  (0, 519)	1
  (307, 0)	1.0
  (616, 0)	2.0
  (2325, 0)	1.0
  (2887, 0)	1.0
  (3147, 0)	1.0
  (4283, 0)	1.0
  (4426, 0)	1.0
  (5778, 0)	4.0
  (6240, 0)	1.0
  (7629, 0)	1.0
  (8458, 0)	1.0
  (8657, 0)	1.0
  (8757, 0)	1.0
  (9066, 0)	1.0
  (11814, 0)	1.0
  (12407, 0)	1.0
  (14728, 0)	1.0
  (15555, 0)	1.0
  (16381, 0)	1.0
  (17225, 0)	1.0
  (18320, 0)	1.0
  (19663